# HHL algorithm

In [1]:
from qiskit import QuantumRegister, QuantumCircuit
import numpy as np
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
from qiskit import IBMQ, transpile

# for mitagating readout errors
from qiskit.utils.mitigation import complete_meas_cal


In [2]:
def HHL_2(diag: float, offdiag: float, theta: float, t: float):

    NUM_QUBITS = 4  # Total number of qubits
    nb = 1  # Number of qubits representing the solution
    nl = 2  # Number of qubits representing the eigenvalues

    a = diag  # Matrix diagonal
    b = offdiag  # Matrix off-diagonal

    # Initialize the quantum and classical registers
    qr = QuantumRegister(NUM_QUBITS)

    # Create a Quantum Circuit
    qc = QuantumCircuit(qr)

    qrb = qr[0:nb]
    qrl = qr[nb:nb+nl]
    qra = qr[nb+nl:nb+nl+1]

    # State preparation.
    qc.ry(2*theta, qrb[0])

    # QPE with e^{iAt}
    for qu in qrl:
        qc.h(qu)

    qc.p(a*t, qrl[0])
    qc.p(a*t*2, qrl[1])

    qc.u(b*t, -np.pi/2, np.pi/2, qrb[0])


    # Controlled e^{iAt} on \lambda_{1}:
    params=b*t

    qc.p(np.pi/2,qrb[0])
    qc.cx(qrl[0],qrb[0])
    qc.ry(params,qrb[0])
    qc.cx(qrl[0],qrb[0])
    qc.ry(-params,qrb[0])
    qc.p(3*np.pi/2,qrb[0])

    # Controlled e^{2iAt} on \lambda_{2}:
    params = b*t*2

    qc.p(np.pi/2,qrb[0])
    qc.cx(qrl[1],qrb[0])
    qc.ry(params,qrb[0])
    qc.cx(qrl[1],qrb[0])
    qc.ry(-params,qrb[0])
    qc.p(3*np.pi/2,qrb[0])

    # Inverse QFT
    qc.h(qrl[1])
    qc.rz(-np.pi/4,qrl[1])
    qc.cx(qrl[0],qrl[1])
    qc.rz(np.pi/4,qrl[1])
    qc.cx(qrl[0],qrl[1])
    qc.rz(-np.pi/4,qrl[0])
    qc.h(qrl[0])

    # Eigenvalue rotation
    t1=(-np.pi +np.pi/3 - 2*np.arcsin(1/3))/4
    t2=(-np.pi -np.pi/3 + 2*np.arcsin(1/3))/4
    t3=(np.pi -np.pi/3 - 2*np.arcsin(1/3))/4
    t4=(np.pi +np.pi/3 + 2*np.arcsin(1/3))/4

    qc.cx(qrl[1],qra[0])
    qc.ry(t1,qra[0])
    qc.cx(qrl[0],qra[0])
    qc.ry(t2,qra[0])
    qc.cx(qrl[1],qra[0])
    qc.ry(t3,qra[0])
    qc.cx(qrl[0],qra[0])
    qc.ry(t4,qra[0])
    qc.measure_all()

    return qc

    # print(f"Depth: {qc.depth()}")
    # print(f"CNOTS: {qc.count_ops()['cx']}")
    # qc.draw(fold=-1)

In [3]:
def extract_counts(dict):
    total_counts = 0
    relevant_count0 = 0
    relevant_count1 = 0
    for key in dict:
        if key[0] == '0':
            if key[-1] == '1':
                relevant_count1 += dict[key]
            if key[-1] == "0":
                relevant_count0 += dict[key]
        total_counts += dict[key]
    return (relevant_count0/total_counts, relevant_count1/total_counts)

In [4]:
def extract_solution(probs):
    return (np.sqrt(probs[0]), np.sqrt(probs[1]))

In [5]:
def calculate_normalized_fidelity(v1, v2):
    return np.dot(np.array(v1)/np.linalg.norm(v1), np.array(v2)/np.linalg.norm(v2))

In [ ]:

# load IBM account
IBMQ.save_account('2bb24b1ffb16645433661cd2214aedc8d53fef6a635ac74857b0e282de4e4597754228444daece952006fb5dff87b434835547e926a5539b24f800818c08e394',overwrite=True)
IBMQ.load_account()

provider = IBMQ.providers()
provider = IBMQ.get_provider(hub='ibm-q-education', group='harvard', project='qse-210')
backend=provider.get_backend("ibmq_belem")